In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
from shared import PATH_SLIIDERS_ECON, PATH_SLIIDERS_ECON_SEG

In [3]:
SEG_CHUNKSIZE = 100

In [4]:
sliiders = xr.open_zarr(PATH_SLIIDERS_ECON).load()

# clean and cast for double-precision math
for v in sliiders.variables:
    sliiders[v].encoding = {}
    if sliiders[v].dtype == "float32":
        sliiders[v] = sliiders[v].astype("float64")

In [5]:
ref_ypcc = sliiders.ypcc.sel(country="USA", drop=True).rename("ref_income")

grouper = sliiders.seg
usa_ypcc_ref = (
    sliiders.ypcc.sel(country="USA", drop=True).load().reset_coords(drop=True)
)

out = (
    sliiders[["K_2019", "pop_2019", "landarea", "length", "wetland"]]
    .groupby(grouper)
    .sum("seg_adm")
)

out[["surge_height", "gumbel_params", "SLR_site_id"]] = (
    sliiders[["surge_height", "gumbel_params", "SLR_site_id"]].groupby(grouper).first()
)


def weighted_avg(varname, wts_in):
    if "seg_adm" not in sliiders[varname].dims:
        sliiders[varname] = sliiders[varname].sel(country=sliiders.seg_country)
    wts_out = wts_in.groupby(grouper).sum("seg_adm")
    unweighted = sliiders[varname].groupby(grouper).sum("seg_adm")
    out[varname] = (
        (sliiders[varname] * wts_in).groupby(grouper).sum("seg_adm") / wts_out
    ).where(wts_out > 0, unweighted)


for v, w in [
    (
        "mobcapfrac",
        sliiders.K_2019.sum("elev"),
    ),
    ("pop_scale", sliiders.pop_2019.sum("elev")),
    ("K_scale", sliiders.K_2019.sum("elev")),
    ("interior", sliiders.landarea.sum("elev")),
    ("pc", sliiders.length),
    ("ypcc", sliiders.pop_2019.sum("elev")),
    ("wetlandservice", sliiders.wetland.sum("elev")),
]:
    weighted_avg(v, w)

out["rho"] = out.ypcc / (out.ypcc + usa_ypcc_ref.sel(year=2000, drop=True))

# merge on non-aggregated vars
other_vars = sliiders[
    [
        d
        for d in sliiders.data_vars
        if "seg_adm" not in sliiders[d].dims and d not in out.data_vars
    ]
]
out = xr.merge((out, other_vars))
out["ref_income"] = ref_ypcc

In [6]:
out.chunk({"seg": SEG_CHUNKSIZE}).to_zarr(PATH_SLIIDERS_ECON_SEG, mode="w")